<h1>preprocessing<h1>

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [76]:
random_state=42

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [23]:
# read and filter

path = '../data/scraped.csv'
df = pd.read_csv(path)

# filter out for posts w/ >1 comments
df = df[df['num_comments'] > 1]

# filter out these (uninterpretable cols)
df.drop(columns=['title', 'url', 'upvotes', 'num_comments', 'symmetry_td'], inplace=True)

<h1>splitting<h1>

In [64]:
ohe = OneHotEncoder(sparse_output=False)
ss = StandardScaler()

In [37]:
# split by subreddit (might consider doing above)

df_itap = df[df['subreddit'] == 'itookapicture']
df_pc = df[df['subreddit'] == 'photocritique']
df_post = df[df['subreddit'] == 'postprocessing']

# drop the subred col
df_itap.drop(columns='subreddit', inplace=True)
df_pc.drop(columns='subreddit', inplace=True)
df_post.drop(columns='subreddit', inplace=True)

dfs = [df_itap, df_pc, df_post]


/var/folders/1c/h8d9s2m12xl44x37yf7xzw5h0000gn/T/ipykernel_1292/3433872702.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itap.drop(columns='subreddit', inplace=True)
/var/folders/1c/h8d9s2m12xl44x37yf7xzw5h0000gn/T/ipykernel_1292/3433872702.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pc.drop(columns='subreddit', inplace=True)
/var/folders/1c/h8d9s2m12xl44x37yf7xzw5h0000gn/T/ipykernel_1292/3433872702.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [41]:
# points after preprocessing
print(f'itap: {len(df_itap)}, photocritique: {len(df_pc)}, postprocess: {len(df_post)}')

itap: 1307, photocritique: 1663, postprocess: 163


In [46]:
df_itap.columns.to_list()

['upvote_ratio',
 'vote-to-comment',
 'brightness',
 'contrast',
 'saturation',
 'symmetry_lr',
 'light_balance_lr',
 'light_balance_td',
 'hue_diversity',
 'dynamic_range',
 'sharpness',
 'thirds_balance',
 'texture_variance',
 'subject_type',
 'genre',
 'perspective',
 'color_mode',
 'complexity',
 'mean_sentiment']

In [45]:
dfs = [df_itap, df_pc, df_post]

In [60]:
names = ['itap', 'pc', 'post']
df_dic = {}

for name, d in zip(names, dfs):
    df_dic[f'{name}_target'] = d['mean_sentiment']
    df_dic[f'{name}_features'] = d.drop('mean_sentiment', axis=1)

In [61]:
cont = ['upvote_ratio', 'vote-to-comment', 'brightness', 'contrast',
       'saturation', 'symmetry_lr', 'light_balance_lr', 'light_balance_td',
       'hue_diversity', 'dynamic_range', 'sharpness', 'thirds_balance',
       'texture_variance']

cat = ['subject_type', 'genre', 'perspective', 'color_mode', 'complexity']

for key, df_feat in df_dic.items():
    if key.endswith('_features'):
        part_cont = df_feat[cont]
        part_cat = df_feat[cat]
    
        cont_scaled = ss.fit_transform(part_cont)
        cat_scaled = ohe.fit_transform(part_cat)

        cont_scaled = pd.DataFrame(cont_scaled, columns=cont)
        cat_scaled = pd.DataFrame(cat_scaled, columns=ohe.get_feature_names_out(cat))

        df_dic[key] = pd.concat([cont_scaled, cat_scaled], axis=1)

In [63]:
# features after preprocessing, identical across sets
print(f'features: {len(df_dic['itap_features'].columns.unique())}')

features: 34


In [68]:
# unpack
itap_features = df_dic['itap_features']
itap_target = df_dic['itap_target']

pc_features = df_dic['pc_features']
pc_target = df_dic['pc_target']

post_features = df_dic['post_features']
post_target = df_dic['post_target']

In [81]:
# itap splits
itap_X_train, itap_X_other, itap_y_train, itap_y_other = train_test_split(itap_features, itap_target, test_size=0.2, random_state=random_state)
itap_X_test, itap_X_val, itap_y_test, itap_y_val = train_test_split(itap_X_other, itap_y_other, test_size=0.5, random_state=random_state)

# itap_X_train, itap_X_val, itap_X_test, itap_y_train, itap_y_val, itap_y_test


# pc splits
pc_X_train, pc_X_other, pc_y_train, pc_y_other = train_test_split(pc_features, pc_target, test_size=0.2, random_state=random_state)
pc_X_test, pc_X_val, pc_y_test, pc_y_val = train_test_split(pc_X_other, pc_y_other, test_size=0.5, random_state=random_state)

# pc_X_train, pc_X_val, pc_X_test, pc_y_train, pc_y_val, pc_y_test


# post splits (has the least)
post_X_train, post_X_other, post_y_train, post_y_other = train_test_split(post_features, post_target, test_size=0.1, random_state=random_state)
post_X_test, post_X_val, post_y_test, post_y_val = train_test_split(post_X_other, post_y_other, test_size=0.5, random_state=random_state)

# post_X_train, post_X_val, post_X_test, post_y_train, post_y_val, post_y_test

(     upvote_ratio  vote-to-comment  brightness  contrast  saturation  \
 105      0.372506        -0.348704    0.872611  1.625194   -0.397460   
 151      1.051178         1.514863    0.547833 -0.223596    0.420595   
 16      -1.437286        -0.522250   -0.444292  0.782833   -0.036737   
 75       0.960689        -0.419639   -0.131610 -0.485519    0.933142   
 18      -0.984838        -0.513404    0.414825  0.486411   -0.550074   
 ..            ...              ...         ...       ...         ...   
 71       1.096423        -0.504558    0.227483  0.772467   -1.812864   
 106      1.005934        -0.403362    0.956703  0.754767   -1.905454   
 14      -1.346796        -0.522250    0.922928  1.322848   -0.565167   
 92       1.005934        -0.016479   -0.718690 -1.450475   -0.162618   
 102      1.096423        -0.008659    0.130321  0.710007   -0.816519   
 
      symmetry_lr  light_balance_lr  light_balance_td  hue_diversity  \
 105     0.562901          1.124643          1.479